<a href="https://colab.research.google.com/github/ADRIANVM117/data-science-portfolio/blob/main/Backtesting/backtesting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip uninstall bokeh -y
!pip install bokeh==2.4.3


Found existing installation: bokeh 3.6.2
Uninstalling bokeh-3.6.2:
  Successfully uninstalled bokeh-3.6.2
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.5/18.5 MB 47.5 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
holoviews 1.20.0 requires bokeh>=3.1, but you have bokeh 2.4.3 which is incompatible.
panel 1.5.4 requires bokeh<3.7.0,>=3.5.0, but you have bokeh 2.4.3 which is incompatible.


In [ ]:
pip install backtesting


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 175.5/175.5 kB 5.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for backtesting: filename=Backtesting-0.3.3-py3-none-any.whl size=173912 sha256=deb5861fc27b82b293d9ba3fbdff707233f37aac3fa7e94933af7c6720046482
  Stored in directory: /root/.cache/pip/wheels/e2/30/7f/19cbe31987c6ebdb47f1f510343249066711609e3da2d57176
Successfully built backtesting


In [ ]:
!pip install backtesting


In [ ]:
!pip show backtesting


Name: Backtesting
Version: 0.3.3
Summary: Backtest trading strategies in Python
Home-page: https://kernc.github.io/backtesting.py/
Author: Zach Lûster
Author-email: 
License: AGPL-3.0
Location: /usr/local/lib/python3.10/dist-packages
Requires: bokeh, numpy, pandas
Required-by: 


In [ ]:
# Descargar la librería TA-Lib
!wget http://prdownloads.sourceforge.net/ta-lib/ta-lib-0.4.0-src.tar.gz
!tar xvzf ta-lib-0.4.0-src.tar.gz
%cd ta-lib
!./configure --prefix=/usr
!make
!make install
%cd ..


--2024-12-27 10:46:44--  http://prdownloads.sourceforge.net/ta-lib/ta-lib-0.4.0-src.tar.gz
Resolving prdownloads.sourceforge.net (prdownloads.sourceforge.net)... 104.18.12.149, 104.18.13.149, 2606:4700::6812:d95, ...
Connecting to prdownloads.sourceforge.net (prdownloads.sourceforge.net)|104.18.12.149|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: http://downloads.sourceforge.net/project/ta-lib/ta-lib/0.4.0/ta-lib-0.4.0-src.tar.gz [following]
--2024-12-27 10:46:44--  http://downloads.sourceforge.net/project/ta-lib/ta-lib/0.4.0/ta-lib-0.4.0-src.tar.gz
Resolving downloads.sourceforge.net (downloads.sourceforge.net)... 104.18.13.149, 104.18.12.149, 2606:4700::6812:c95, ...
Reusing existing connection to prdownloads.sourceforge.net:80.
HTTP request sent, awaiting response... 302 Found
Location: http://cfhcable.dl.sourceforge.net/project/ta-lib/ta-lib/0.4.0/ta-lib-0.4.0-src.tar.gz?viasf=1 [following]
--2024-12-27 10:46:44--  http://cfhcable.dl.sour

In [ ]:
!pip install TA-Lib


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 372.0/372.0 kB 8.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for TA-Lib: filename=ta_lib-0.5.2-cp310-cp310-linux_x86_64.whl size=2418218 sha256=3f5005e263157887743c45414502458bab721e1b5af44bc8d841f46f61e0c02b
  Stored in directory: /root/.cache/pip/wheels/02/10/45/dc1db64a6e8eafb6553e43d1dd2d60603d544dd785def1f4ed
Successfully built TA-Lib


In [ ]:
import talib
import pandas as pd
import yfinance as yf
from backtesting import Backtest, Strategy
from backtesting.lib import crossover


/usr/local/lib/python3.10/dist-packages/backtesting/_plotting.py:50: UserWarning: Jupyter Notebook detected. Setting Bokeh output to notebook. This may not work in Jupyter clients without JavaScript support (e.g. PyCharm, Spyder IDE). Reset with `backtesting.set_bokeh_output(notebook=False)`.
  warnings.warn('Jupyter Notebook detected. '


In [ ]:
from backtesting import Backtest, Strategy
from backtesting.lib import crossover
import yfinance as yf
import talib
import pandas as pd
import numpy as np
from bokeh.io import output_notebook

# Configurar salida en notebooks
output_notebook()

# Descargar datos históricos de Chedraui
data = yf.download("CHDRAUIB.MX", start="2023-01-01", end="2024-12-26")

data = data[['Open', 'High', 'Low', 'Close', 'Volume']]
data.columns = ['Open', 'High', 'Low', 'Close', 'Volume']
data.index = pd.to_datetime(data.index)

# Calcular componentes de la nube de Ichimoku
data['Tenkan_sen'] = (data['High'].rolling(window=9).max() + data['Low'].rolling(window=9).min()) / 2
data['Kijun_sen'] = (data['High'].rolling(window=26).max() + data['Low'].rolling(window=26).min()) / 2
data['Senkou_Span_A'] = ((data['Tenkan_sen'] + data['Kijun_sen']) / 2).shift(26)
data['Senkou_Span_B'] = ((data['High'].rolling(window=52).max() + data['Low'].rolling(window=52).min()) / 2).shift(26)
data['Chikou_Span'] = data['Close'].shift(-26)

class Ichimoku_Strategy(Strategy):
    def init(self):
        # Indicadores Ichimoku
        self.tenkan_sen = self.I(lambda x: x, data['Tenkan_sen'])
        self.kijun_sen = self.I(lambda x: x, data['Kijun_sen'])
        self.senkou_span_a = self.I(lambda x: x, data['Senkou_Span_A'])
        self.senkou_span_b = self.I(lambda x: x, data['Senkou_Span_B'])
        self.chikou_span = self.I(lambda x: x, data['Chikou_Span'])

    def next(self):
        # Estrategia basada en la nube de Ichimoku
        if (self.data.Close[-1] > self.senkou_span_a[-1] and
            self.data.Close[-1] > self.senkou_span_b[-1] and
            self.tenkan_sen[-1] > self.kijun_sen[-1] and
            self.chikou_span[-1] > self.data.Close[-1]):
            self.buy()
        elif (self.data.Close[-1] < self.senkou_span_a[-1] and
              self.data.Close[-1] < self.senkou_span_b[-1] and
              self.tenkan_sen[-1] < self.kijun_sen[-1] and
              self.chikou_span[-1] < self.data.Close[-1]):
            self.sell()

bt = Backtest(data, Ichimoku_Strategy, cash=100000, exclusive_orders=True)
stats_ichimoku = bt.run()
print(stats_ichimoku)
bt.plot()


[*********************100%***********************]  1 of 1 completed
/usr/local/lib/python3.10/dist-packages/backtesting/_plotting.py:455: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  df2 = (df.assign(_width=1).set_index('datetime')


Start                     2023-01-02 00:00:00
End                       2024-12-24 00:00:00
Duration                    722 days 00:00:00
Exposure Time [%]                   80.522088
Equity Final [$]                144319.194878
Equity Peak [$]                 147182.211227
Return [%]                          44.319195
Buy & Hold Return [%]               50.939536
Return (Ann.) [%]                    20.39871
Volatility (Ann.) [%]               25.778657
Sharpe Ratio                         0.791302
Sortino Ratio                        1.419464
Calmar Ratio                         1.561445
Max. Drawdown [%]                  -13.063991
Avg. Drawdown [%]                   -6.425661
Max. Drawdown Duration      245 days 00:00:00
Avg. Drawdown Duration       55 days 00:00:00
# Trades                                  104
Win Rate [%]                        55.769231
Best Trade [%]                       5.322686
Worst Trade [%]                     -6.439433
Avg. Trade [%]                    

Row(id='13132', ...)

In [ ]:
from backtesting import Backtest, Strategy
from backtesting.lib import crossover
import yfinance as yf
import talib
import pandas as pd
import numpy as np
from bokeh.io import output_notebook

# Configurar salida en notebooks
output_notebook()

# Descargar datos históricos de Chedraui
data = yf.download("CHDRAUIB.MX", start="2023-01-01", end="2024-12-26")

data = data[['Open', 'High', 'Low', 'Close', 'Volume']]
data.columns = ['Open', 'High', 'Low', 'Close', 'Volume']
data.index = pd.to_datetime(data.index)

# Calcular componentes de la nube de Ichimoku
data['Tenkan_sen'] = (data['High'].rolling(window=9).max() + data['Low'].rolling(window=9).min()) / 2
data['Kijun_sen'] = (data['High'].rolling(window=26).max() + data['Low'].rolling(window=26).min()) / 2
data['Senkou_Span_A'] = ((data['Tenkan_sen'] + data['Kijun_sen']) / 2).shift(26)
data['Senkou_Span_B'] = ((data['High'].rolling(window=52).max() + data['Low'].rolling(window=52).min()) / 2).shift(26)
data['Chikou_Span'] = data['Close'].shift(-26)

class Ichimoku_ATR_MACD_Strategy(Strategy):
    def init(self):
        # Indicadores Ichimoku
        self.tenkan_sen = self.I(lambda x: x, data['Tenkan_sen'])
        self.kijun_sen = self.I(lambda x: x, data['Kijun_sen'])
        self.senkou_span_a = self.I(lambda x: x, data['Senkou_Span_A'])
        self.senkou_span_b = self.I(lambda x: x, data['Senkou_Span_B'])
        self.chikou_span = self.I(lambda x: x, data['Chikou_Span'])

        # Indicador MACD
        self.macd, self.macd_signal, _ = self.I(talib.MACD, self.data.Close, 12, 26, 9)

        # Indicador ATR para ajustar el riesgo
        self.atr = self.I(talib.ATR, self.data.High, self.data.Low, self.data.Close, 14)

    def next(self):
        # Gestión de riesgo dinámica basada en ATR
        for trade in self.trades:
            trade.sl = trade.entry_price - 2 * self.atr[-1]  # Stop-loss a 2 veces el ATR por debajo
            trade.tp = trade.entry_price + 3 * self.atr[-1]  # Take-profit a 3 veces el ATR por encima

        # Estrategia combinada
        if (self.data.Close[-1] > self.senkou_span_a[-1] and
            self.data.Close[-1] > self.senkou_span_b[-1] and
            self.tenkan_sen[-1] > self.kijun_sen[-1] and
            self.chikou_span[-1] > self.data.Close[-1] and
            self.macd[-1] > self.macd_signal[-1]):  # Confirmación con MACD
            self.buy()

        elif (self.data.Close[-1] < self.senkou_span_a[-1] and
              self.data.Close[-1] < self.senkou_span_b[-1] and
              self.tenkan_sen[-1] < self.kijun_sen[-1] and
              self.chikou_span[-1] < self.data.Close[-1] and
              self.macd[-1] < self.macd_signal[-1]):  # Confirmación con MACD
            self.sell()

bt = Backtest(data, Ichimoku_ATR_MACD_Strategy, cash=100000, exclusive_orders=True)
stats_combined = bt.run()
print(stats_combined)
bt.plot()


[*********************100%***********************]  1 of 1 completed
/usr/local/lib/python3.10/dist-packages/backtesting/_plotting.py:455: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  df2 = (df.assign(_width=1).set_index('datetime')


Start                     2023-01-02 00:00:00
End                       2024-12-24 00:00:00
Duration                    722 days 00:00:00
Exposure Time [%]                    33.13253
Equity Final [$]                155184.850799
Equity Peak [$]                 162953.728747
Return [%]                          55.184851
Buy & Hold Return [%]               50.939536
Return (Ann.) [%]                   24.903428
Volatility (Ann.) [%]               16.650668
Sharpe Ratio                         1.495641
Sortino Ratio                        3.377596
Calmar Ratio                          2.07189
Max. Drawdown [%]                  -12.019664
Avg. Drawdown [%]                   -3.801098
Max. Drawdown Duration      126 days 00:00:00
Avg. Drawdown Duration       30 days 00:00:00
# Trades                                   69
Win Rate [%]                        60.869565
Best Trade [%]                       7.594003
Worst Trade [%]                     -5.125307
Avg. Trade [%]                    

Row(id='17764', ...)

Optimización de Parámetros:

Ajustar parámetros de Ichimoku, ATR, y MACD para maximizar el rendimiento.
Ejemplo: Probar diferentes valores para los períodos de ATR (14) o los períodos rápidos/lentos de MACD (12, 26, 9).
Validar Robustez:

Realizar pruebas en diferentes marcos temporales (por ejemplo, 4H, 1D) y mercados (otras acciones o índices).
Incorporar Indicadores Complementarios:

Volumen: Agregar indicadores como OBV para confirmar la fuerza de las tendencias.
Tendencia Adicional: Combinar con Parabolic SAR para identificar puntos de reversión.
Optimización de Gestión de Riesgo:

Ajustar la relación riesgo/beneficio (por ejemplo, 1.5x ATR para SL y 2x ATR para TP).
Backtesting Extensivo:

Analizar métricas adicionales como Sharpe Ratio, Max Drawdown, y Profit Factor para evaluar la calidad de la estrategia.

In [ ]:
from backtesting import Backtest, Strategy
from backtesting.lib import crossover
import yfinance as yf
import talib
import pandas as pd
import numpy as np
from bokeh.io import output_notebook

# Configurar salida en notebooks
output_notebook()

# Descargar datos históricos de Chedraui
data = yf.download("CHDRAUIB.MX", start="2023-01-01", end="2024-12-26")

data = data[['Open', 'High', 'Low', 'Close', 'Volume']]
data.columns = ['Open', 'High', 'Low', 'Close', 'Volume']
data.index = pd.to_datetime(data.index)

# Calcular componentes de la nube de Ichimoku
data['Tenkan_sen'] = (data['High'].rolling(window=9).max() + data['Low'].rolling(window=9).min()) / 2
data['Kijun_sen'] = (data['High'].rolling(window=26).max() + data['Low'].rolling(window=26).min()) / 2
data['Senkou_Span_A'] = ((data['Tenkan_sen'] + data['Kijun_sen']) / 2).shift(26)
data['Senkou_Span_B'] = ((data['High'].rolling(window=52).max() + data['Low'].rolling(window=52).min()) / 2).shift(26)
data['Chikou_Span'] = data['Close'].shift(-26)

class Ichimoku_ATR_MACD_PSAR_Strategy(Strategy):
    # Definir parámetros como atributos de clase
    atr_period = 14
    macd_fast = 12
    macd_slow = 26
    macd_signal = 9

    def init(self):
        # Indicadores Ichimoku
        self.tenkan_sen = self.I(lambda x: x, data['Tenkan_sen'])
        self.kijun_sen = self.I(lambda x: x, data['Kijun_sen'])
        self.senkou_span_a = self.I(lambda x: x, data['Senkou_Span_A'])
        self.senkou_span_b = self.I(lambda x: x, data['Senkou_Span_B'])
        self.chikou_span = self.I(lambda x: x, data['Chikou_Span'])

        # Indicador MACD
        self.macd, self.macd_signal, _ = self.I(
            talib.MACD, self.data.Close, self.macd_fast, self.macd_slow, self.macd_signal)

        # Indicador ATR para ajustar el riesgo
        self.atr = self.I(talib.ATR, self.data.High, self.data.Low, self.data.Close, self.atr_period)

        # Indicador Parabolic SAR
        self.psar = self.I(talib.SAR, self.data.High, self.data.Low, acceleration=0.02, maximum=0.2)

    def next(self):
        # Gestión de riesgo dinámica basada en ATR
        for trade in self.trades:
            trade.sl = trade.entry_price - 2 * self.atr[-1]  # Stop-loss a 2 veces el ATR por debajo
            trade.tp = trade.entry_price + 3 * self.atr[-1]  # Take-profit a 3 veces el ATR por encima

        # Estrategia combinada
        if (self.data.Close[-1] > self.senkou_span_a[-1] and
            self.data.Close[-1] > self.senkou_span_b[-1] and
            self.tenkan_sen[-1] > self.kijun_sen[-1] and
            self.chikou_span[-1] > self.data.Close[-1] and
            self.macd[-1] > self.macd_signal[-1] and
            self.data.Close[-1] > self.psar[-1]):  # Confirmación con PSAR indicando tendencia alcista
            self.buy()

        elif (self.data.Close[-1] < self.senkou_span_a[-1] and
              self.data.Close[-1] < self.senkou_span_b[-1] and
              self.tenkan_sen[-1] < self.kijun_sen[-1] and
              self.chikou_span[-1] < self.data.Close[-1] and
              self.macd[-1] < self.macd_signal[-1] and
              self.data.Close[-1] < self.psar[-1]):  # Confirmación con PSAR indicando tendencia bajista
            self.sell()

# Rango de parámetros a optimizar
optimize_ranges = {
    'atr_period': range(10, 21, 2),
    'macd_fast': range(10, 15, 2),
    'macd_slow': range(24, 31, 2),
    'macd_signal': range(8, 13, 2)
}

# Configuración del backtest
bt = Backtest(data, Ichimoku_ATR_MACD_PSAR_Strategy, cash=100000, exclusive_orders=True)

# Ejecutar optimización
stats_optimized = bt.optimize(
    **optimize_ranges,
    maximize='Return [%]',
    constraint=lambda p: p.macd_fast < p.macd_slow
)

# Resultados de la mejor configuración
print(stats_optimized)
bt.plot()


[*********************100%***********************]  1 of 1 completed


Backtest.optimize:   0%|          | 0/2 [00:00<?, ?it/s]

Start                     2023-01-02 00:00:00
End                       2024-12-24 00:00:00
Duration                    722 days 00:00:00
Exposure Time [%]                   32.730924
Equity Final [$]                162051.429177
Equity Peak [$]                 167704.756749
Return [%]                          62.051429
Buy & Hold Return [%]               50.939536
Return (Ann.) [%]                    27.67016
Volatility (Ann.) [%]               17.137596
Sharpe Ratio                         1.614588
Sortino Ratio                         3.83932
Calmar Ratio                         2.293796
Max. Drawdown [%]                  -12.063044
Avg. Drawdown [%]                   -3.917219
Max. Drawdown Duration      126 days 00:00:00
Avg. Drawdown Duration       29 days 00:00:00
# Trades                                   62
Win Rate [%]                        61.290323
Best Trade [%]                        8.09442
Worst Trade [%]                     -4.895846
Avg. Trade [%]                    

/usr/local/lib/python3.10/dist-packages/backtesting/_plotting.py:455: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  df2 = (df.assign(_width=1).set_index('datetime')


Row(id='22202', ...)

In [ ]:
from backtesting import Backtest, Strategy
from backtesting.lib import crossover
import yfinance as yf
import talib
import pandas as pd
import numpy as np
from bokeh.io import output_notebook

# Configurar salida en notebooks
output_notebook()

# Descargar datos históricos de Chedraui
data = yf.download("CHDRAUIB.MX", start="2023-01-01", end="2024-12-26")

data = data[['Open', 'High', 'Low', 'Close', 'Volume']]
data.columns = ['Open', 'High', 'Low', 'Close', 'Volume']
data.index = pd.to_datetime(data.index)

# Calcular componentes de la nube de Ichimoku
data['Tenkan_sen'] = (data['High'].rolling(window=9).max() + data['Low'].rolling(window=9).min()) / 2
data['Kijun_sen'] = (data['High'].rolling(window=26).max() + data['Low'].rolling(window=26).min()) / 2
data['Senkou_Span_A'] = ((data['Tenkan_sen'] + data['Kijun_sen']) / 2).shift(26)
data['Senkou_Span_B'] = ((data['High'].rolling(window=52).max() + data['Low'].rolling(window=52).min()) / 2).shift(26)
data['Chikou_Span'] = data['Close'].shift(-26)

class Ichimoku_ATR_MACD_PSAR_Strategy(Strategy):
    # Definir parámetros como atributos de clase
    atr_period = 14
    macd_fast = 12
    macd_slow = 26
    macd_signal = 9
    risk_reward_ratio_sl = 1.5  # Relación para Stop-Loss
    risk_reward_ratio_tp = 2.0  # Relación para Take-Profit

    def init(self):
        # Indicadores Ichimoku
        self.tenkan_sen = self.I(lambda x: x, data['Tenkan_sen'])
        self.kijun_sen = self.I(lambda x: x, data['Kijun_sen'])
        self.senkou_span_a = self.I(lambda x: x, data['Senkou_Span_A'])
        self.senkou_span_b = self.I(lambda x: x, data['Senkou_Span_B'])
        self.chikou_span = self.I(lambda x: x, data['Chikou_Span'])

        # Indicador MACD
        self.macd, self.macd_signal, _ = self.I(
            talib.MACD, self.data.Close, self.macd_fast, self.macd_slow, self.macd_signal)

        # Indicador ATR para ajustar el riesgo
        self.atr = self.I(talib.ATR, self.data.High, self.data.Low, self.data.Close, self.atr_period)

        # Indicador Parabolic SAR
        self.psar = self.I(talib.SAR, self.data.High, self.data.Low, acceleration=0.02, maximum=0.2)

    def next(self):
        # Gestión de riesgo dinámica basada en ATR y relaciones riesgo/beneficio
        for trade in self.trades:
            trade.sl = trade.entry_price - self.risk_reward_ratio_sl * self.atr[-1]  # Stop-loss dinámico
            trade.tp = trade.entry_price + self.risk_reward_ratio_tp * self.atr[-1]  # Take-profit dinámico

        # Estrategia combinada
        if (self.data.Close[-1] > self.senkou_span_a[-1] and
            self.data.Close[-1] > self.senkou_span_b[-1] and
            self.tenkan_sen[-1] > self.kijun_sen[-1] and
            self.chikou_span[-1] > self.data.Close[-1] and
            self.macd[-1] > self.macd_signal[-1] and
            self.data.Close[-1] > self.psar[-1]):  # Confirmación con PSAR indicando tendencia alcista
            self.buy()

        elif (self.data.Close[-1] < self.senkou_span_a[-1] and
              self.data.Close[-1] < self.senkou_span_b[-1] and
              self.tenkan_sen[-1] < self.kijun_sen[-1] and
              self.chikou_span[-1] < self.data.Close[-1] and
              self.macd[-1] < self.macd_signal[-1] and
              self.data.Close[-1] < self.psar[-1]):  # Confirmación con PSAR indicando tendencia bajista
            self.sell()

# Rango de parámetros a optimizar
optimize_ranges = {
    'atr_period': range(10, 21, 2),
    'macd_fast': range(10, 15, 2),
    'macd_slow': range(24, 31, 2),
    'macd_signal': range(8, 13, 2),
    'risk_reward_ratio_sl': [1.0, 1.5, 2.0],
    'risk_reward_ratio_tp': [1.5, 2.0, 2.5]
}

# Configuración del backtest
bt = Backtest(data, Ichimoku_ATR_MACD_PSAR_Strategy, cash=100000, exclusive_orders=True)

# Ejecutar optimización
stats_optimized = bt.optimize(
    **optimize_ranges,
    maximize='Return [%]',
    constraint=lambda p: p.risk_reward_ratio_sl < p.risk_reward_ratio_tp
)

# Resultados de la mejor configuración
print(stats_optimized)
bt.plot()


[*********************100%***********************]  1 of 1 completed
/usr/local/lib/python3.10/dist-packages/backtesting/backtesting.py:1488: UserWarning: Searching for best of 1296 configurations.
  output = _optimize_grid()


Backtest.optimize:   0%|          | 0/5 [00:00<?, ?it/s]

Start                     2023-01-02 00:00:00
End                       2024-12-24 00:00:00
Duration                    722 days 00:00:00
Exposure Time [%]                   30.522088
Equity Final [$]                176085.419773
Equity Peak [$]                 176085.419773
Return [%]                           76.08542
Buy & Hold Return [%]               50.939536
Return (Ann.) [%]                   33.150242
Volatility (Ann.) [%]                16.36989
Sharpe Ratio                         2.025074
Sortino Ratio                        5.425851
Calmar Ratio                          2.98176
Max. Drawdown [%]                  -11.117674
Avg. Drawdown [%]                   -2.808555
Max. Drawdown Duration      125 days 00:00:00
Avg. Drawdown Duration       20 days 00:00:00
# Trades                                   66
Win Rate [%]                        60.606061
Best Trade [%]                        6.74535
Worst Trade [%]                     -3.961082
Avg. Trade [%]                    

/usr/local/lib/python3.10/dist-packages/backtesting/_plotting.py:455: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  df2 = (df.assign(_width=1).set_index('datetime')


Row(id='23297', ...)

In [ ]:
# Imprimir métricas clave de rendimiento
print("=== Métricas Adicionales ===")
print(f"Sharpe Ratio: {stats_optimized['Sharpe Ratio']:.2f}")
print(f"Max Drawdown [%]: {stats_optimized['Max. Drawdown [%]']:.2f}")
print(f"Profit Factor: {stats_optimized['Profit Factor']:.2f}")
print(f"Return [%]: {stats_optimized['Return [%]']:.2f}")
print(f"Buy & Hold Return [%]: {stats_optimized['Buy & Hold Return [%]']:.2f}")
print(f"Exposure Time [%]: {stats_optimized['Exposure Time [%]']:.2f}")
print(f"# Trades: {stats_optimized['# Trades']}")

# Análisis de métricas clave
if stats_optimized['Sharpe Ratio'] > 2:
    print("El Sharpe Ratio indica un buen rendimiento ajustado al riesgo.")
else:
    print("El Sharpe Ratio podría mejorar; considere reducir la volatilidad.")

if stats_optimized['Profit Factor'] > 2:
    print("El Profit Factor es excelente, lo que indica que las ganancias superan ampliamente las pérdidas.")
else:
    print("El Profit Factor podría mejorarse, considere optimizar las entradas y salidas.")

if stats_optimized['Max. Drawdown [%]'] < -10:
    print("El Max Drawdown está dentro de un rango aceptable.")
else:
    print("El Max Drawdown es alto, considere ajustar el stop-loss o reducir el tamaño de las posiciones.")


=== Métricas Adicionales ===
Sharpe Ratio: 2.03
Max Drawdown [%]: -11.12
Profit Factor: 3.23
Return [%]: 76.09
Buy & Hold Return [%]: 50.94
Exposure Time [%]: 30.52
# Trades: 66
El Sharpe Ratio indica un buen rendimiento ajustado al riesgo.
El Profit Factor es excelente, lo que indica que las ganancias superan ampliamente las pérdidas.
El Max Drawdown está dentro de un rango aceptable.


In [ ]:
# Estrategia aplicada a 1H Y 1D
# Descargar datos para diferentes marcos temporales (1H y 1D)
timeframes = {'1H': '1h'}
results = {}

for tf_name, tf_interval in timeframes.items():
    print(f"\nProbando marco temporal: {tf_name}")
    data = get_data("CHDRAUIB.MX", start="2023-01-01", end="2024-12-26", interval=tf_interval)

    # Validar que no esté vacío
    if data.empty:
        print(f"Datos vacíos para el intervalo: {tf_interval}.")
        continue

    # Calcular componentes de la nube de Ichimoku
    data['Tenkan_sen'] = (data['High'].rolling(window=9).max() + data['Low'].rolling(window=9).min()) / 2
    data['Kijun_sen'] = (data['High'].rolling(window=26).max() + data['Low'].rolling(window=26).min()) / 2
    data['Senkou_Span_A'] = ((data['Tenkan_sen'] + data['Kijun_sen']) / 2).shift(26)
    data['Senkou_Span_B'] = ((data['High'].rolling(window=52).max() + data['Low'].rolling(window=52).min()) / 2).shift(26)
    data['Chikou_Span'] = data['Close'].shift(-26)

    # Clase de estrategia
    class Ichimoku_ATR_MACD_PSAR_Strategy(Strategy):
        atr_period = 14
        macd_fast = 12
        macd_slow = 26
        macd_signal = 9

        def init(self):
            self.tenkan_sen = self.I(lambda x: x, data['Tenkan_sen'])
            self.kijun_sen = self.I(lambda x: x, data['Kijun_sen'])
            self.senkou_span_a = self.I(lambda x: x, data['Senkou_Span_A'])
            self.senkou_span_b = self.I(lambda x: x, data['Senkou_Span_B'])
            self.chikou_span = self.I(lambda x: x, data['Chikou_Span'])
            self.macd, self.macd_signal, _ = self.I(
                talib.MACD, self.data.Close, self.macd_fast, self.macd_slow, self.macd_signal)
            self.atr = self.I(talib.ATR, self.data.High, self.data.Low, self.data.Close, self.atr_period)
            self.psar = self.I(talib.SAR, self.data.High, self.data.Low, acceleration=0.02, maximum=0.2)

        def next(self):
            for trade in self.trades:
                trade.sl = trade.entry_price - 2 * self.atr[-1]
                trade.tp = trade.entry_price + 3 * self.atr[-1]

            if (self.data.Close[-1] > self.senkou_span_a[-1] and
                self.data.Close[-1] > self.senkou_span_b[-1] and
                self.tenkan_sen[-1] > self.kijun_sen[-1] and
                self.chikou_span[-1] > self.data.Close[-1] and
                self.macd[-1] > self.macd_signal[-1] and
                self.data.Close[-1] > self.psar[-1]):
                self.buy()
            elif (self.data.Close[-1] < self.senkou_span_a[-1] and
                  self.data.Close[-1] < self.senkou_span_b[-1] and
                  self.tenkan_sen[-1] < self.kijun_sen[-1] and
                  self.chikou_span[-1] < self.data.Close[-1] and
                  self.macd[-1] < self.macd_signal[-1] and
                  self.data.Close[-1] < self.psar[-1]):
                self.sell()

    # Configuración del backtest
    bt = Backtest(data, Ichimoku_ATR_MACD_PSAR_Strategy, cash=100000, exclusive_orders=True)

    # Ejecutar optimización
    stats_optimized = bt.optimize(
        atr_period=range(10, 21, 2),
        macd_fast=range(10, 15, 2),
        macd_slow=range(24, 31, 2),
        macd_signal=range(8, 13, 2),
        maximize='Return [%]',
        constraint=lambda p: p.macd_fast < p.macd_slow
    )

    # Almacenar resultados
    results[tf_name] = stats_optimized
    print(stats_optimized)
    bt.plot()


[*********************100%***********************]  1 of 1 completed


Probando marco temporal: 1H


Backtest.optimize:   0%|          | 0/2 [00:00<?, ?it/s]

Start                     2023-01-02 14:30...
End                       2024-12-24 17:30...
Duration                    722 days 03:00:00
Exposure Time [%]                   33.037131
Equity Final [$]                189254.983836
Equity Peak [$]                 198132.733004
Return [%]                          89.254984
Buy & Hold Return [%]               50.754854
Return (Ann.) [%]                   38.099664
Volatility (Ann.) [%]               20.451044
Sharpe Ratio                         1.862969
Sortino Ratio                        5.366771
Calmar Ratio                         4.452059
Max. Drawdown [%]                   -8.557763
Avg. Drawdown [%]                   -1.240852
Max. Drawdown Duration      115 days 00:00:00
Avg. Drawdown Duration        9 days 20:00:00
# Trades                                  499
Win Rate [%]                         55.11022
Best Trade [%]                       6.640815
Worst Trade [%]                     -5.439394
Avg. Trade [%]                    

/usr/local/lib/python3.10/dist-packages/backtesting/_plotting.py:455: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  df2 = (df.assign(_width=1).set_index('datetime')
<ipython-input-50-5b332ccf54e6>:75: UserWarning: Superimposed OHLC plot matches the original plot. Skipping.
  bt.plot()
